In [243]:
import pandas as pd
import numpy as np
import math
import pickle 
from structures import *
from nltk.corpus import stopwords
from nltk import stem
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/Ian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [244]:
#Write pickle file back to dictionary 
toDict = open(r'index.pkl', 'rb')
docTermIndex = pickle.load(toDict)
toDict.close()

In [279]:
query = 'cryptology or space or religion or politics'

# Query Pre-processing, Stemming, and Expansion

In [280]:
qlist = query.strip().split()
#Remove stop words
modified = [term for term in qlist if term not in stopwords.words('english')]

EXPAND = False
'''Expansion
EXPAND = True
for term in modified:
    syns = wordnet.synsets(term) 
    for i in range(len(syns)):
        #print(syns[i].lemmas()[0].name())
        new.append(syns[i].lemmas()[0].name())
'''''''''
if EXPAND == False:
    new = modified
before_stem = np.unique(new)
ps = stem.PorterStemmer()
after_stem = [ps.stem(word) for word in before_stem]
mQuery = np.unique(after_stem)
mQuery

array(['cryptolog', 'polit', 'religion', 'space'], dtype='<U9')

# Query Index and return documents

In [281]:
#read potentially relevant docs into matrix using tf*idf weights 
qMatrix = pd.DataFrame(np.zeros((0,len(mQuery))), columns = mQuery)

for term in mQuery:
    if term in docTermIndex.keys():
        termInfo = docTermIndex.get(term)
        for occurence in termInfo.occList:
            if occurence.docID not in qMatrix.index:
                toAppend = pd.Series(np.zeros(len(qMatrix.columns)), index = qMatrix.columns, name = occurence.docID)
                toAppend[term] = occurence.count * termInfo.idf
                qMatrix = qMatrix.append(toAppend)
            else:
                qMatrix.loc[occurence.docID,term] = occurence.count * termInfo.idf

In [282]:
#compute tfxidf vector of query 
q_vect = [docTermIndex.get(term).idf for term in qMatrix.columns]

#Get cosine similarities for query 
matrix_norm = np.array([np.linalg.norm(qMatrix.iloc[i]) for i in range(len(qMatrix))])
q_norm = np.linalg.norm(q_vect)
sims = np.dot(qMatrix,q_vect)/(matrix_norm * q_norm)
dists = 1 - sims
idx = np.argsort(dists)

In [283]:
user_docs = qMatrix.iloc[idx[:10]].index 
classes = pd.read_csv('classes.csv',index_col=1)

In [284]:
for i, path in enumerate(user_docs):
    parts = path.split('/')
    group = parts[1]
    file = parts[2]
    print('----{}: File {} in folder {}-----\n'.format(i,file,group))
    with open(path, 'r', errors='ignore') as myfile:
        data = myfile.read()
        art = data
        ind = art.find('\n\n')
        art = art[ind+2:]
        #If article(and not post), re-index again to get rid of tags
        if art[0:10].find('archive') != -1:
            ind = art.find('\n\n')
            art = art[ind+2:]
        mid = len(art)//2
        midmid = mid//2
        print('---------------------------------------------\n')
        print(art[mid:mid+200])
        print('---------------------------------------------\n')
        #print(art[:100])

----0: File 15694 in folder sci.crypt-----

---------------------------------------------


I could go on, but I'm sure you see my point.  I'm a single person,
but I react differently on different groups (admittedly, in part
because I'm a contrarian you always shouts "but, on the other hand
---------------------------------------------

----1: File 15243 in folder sci.crypt-----

---------------------------------------------

l this mathematically?
* Why is the one-time pad secure?
* What's a ciphertext-only attack?
* What's a known-plaintext attack?
* What's a chosen-plaintext attack?
* In mathematical terms, what can you
---------------------------------------------

----2: File 15248 in folder sci.crypt-----

---------------------------------------------

 analytical reasoning, application of mathematical tools, pattern
  finding, patience, determination, and luck. The best available
  textbooks on the subject are the Military Cryptanalytics series
  [
------------------------------

,Unnamed: 0,class
docID,,
61059,3880,20_newsgroups/comp.sys.ibm.pc.hardware
61059,14408,20_newsgroups/sci.space
